# template config

In [ ]:
expt_name = '' # name of your experiment

In [ ]:
import sys
import os


def run_gainchart(eval_data_path, model_score_names, eval_set, expt_name):
    lib_path = "/projects/gds-focus/data/wzhao5/Python3"
    if lib_path not in sys.path:
        sys.path.append(lib_path)
        
    from fast_perf_v5 import fast_perf_v5
    
    eval_result_dir = os.path.dirname(eval_data_path)
    excl_eval_result_path = os.path.join(eval_result_dir, f"gainchart_{expt_name}_{eval_set}.xlsx")
    excl_eval_result_csv_path = os.path.join(eval_result_dir, f"gainchart_{expt_name}_{eval_set}.csv")
    print(f"gainchart result path: {excl_eval_result_path}")
    print(f"gainchart result path (csv): {excl_eval_result_csv_path}")
    
    dim_list = [
        'driver_product',
    ]
    
    args = {
        'dataPath': eval_data_path,
        'delimiter': '\x07',
        'badList': ['driver_is_cc_bad_v1'],
        'scoreList': model_score_names,
        'dimList': dim_list,
        'xWeight': ['driver_dol_usd_amt', 'driver_unit_wgt'],
        'yWeight': ['driver_dol_usd_amt', 'driver_unit_wgt'],
        'weightAlias': ['dol', 'unit'],
        'filterExpr': {
            '$AND': {
                'driver_txn_status': 'Approved',
            },
        },
        'OP': 100,
        'TopOP': 20,
        'outputFile': excl_eval_result_path
    }
    
    print(f'start running gainchart')
    perf = fast_perf_v5(**args)
    perf.run()
    
    print(f'finish running model performance evaluation: {expt_name} on {eval_set}')
    df_out = perf.output.copy()
    df_out.to_csv(excl_eval_result_csv_path, index=False)
    print(f'eval result save to {excl_eval_result_csv_path}')
    
    return perf

# not null filter

In [ ]:
config = {
    'filterExpr': {
        '$AND': {
            'driver_decline_type': 'Approved',
            'driver_capture_usd_amt': {'$notnull': ''},
        },
    },
}

# validate filter expr

knonw issue:
- `$gt $lt` doesenot support string compare



In [ ]:
lib_path = "/projects/gds-focus/data/wzhao5/Python3"
if lib_path not in sys.path:
    sys.path.append(lib_path)
    
from lib import parseFilterExpr

In [ ]:

schema_header = ['txn_status', 'auth_no_capture']
dataline = ['approved', 0]
    
filter_expr = {
    "$and": {
        "txn_status": "approved",
        'auth_no_capture': {
            '$eq': 0,
        }, 
    },
}

print(parseFilterExpr(dataline, schema_header, filter_expr))


# copy fastperf output to csv

In [ ]:
perf = None # fast performance instance


df_out = perf.output.copy()
df_out.to_csv(os.path.join(model_eval_path, f"{model_aame}_gainchart.csv"))